##FIS do robô usando lógica difusa para evitar obstáculos
###Aluna: Paloma Duarte de Lira

In [2]:
!pip install scikit-fuzzy
#n tinha fuzz instalado

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 10.4 MB/s eta 0:00:00


In [16]:
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl

In [10]:
# Definição das variáveis de entrada
dist = ctrl.Antecedent(np.arange(0, 10.1, 0.1), 'dist')
ang = ctrl.Antecedent(np.arange(0, np.pi/2, 0.01), 'ang')

# Definição das variáveis de saída
guinada = ctrl.Consequent(np.arange(0, np.pi/2, 0.01), 'guinada') #o senhor escreveu 'guinhada' no ex e eu fiquei deverás confusa
vel = ctrl.Consequent(np.arange(0, 15.1, 0.1), 'vel')

In [11]:
# Conjuntos difusos das entradas
dist['Pequena'] = fuzz.trapmf(dist.universe, [0, 0, 2.1, 2.5])
dist['Longa'] = fuzz.trapmf(dist.universe, [2.3, 2.6, 5.4, 6])
dist['MuitoLonga'] = fuzz.trapmf(dist.universe, [5.2, 5.8, 10, 10])

ang['Pequeno'] = fuzz.zmf(ang.universe, 0, np.pi/6)
ang['Medio'] = fuzz.pimf(ang.universe, np.pi/12, np.pi/4, np.pi/3, np.pi/2)
ang['Amplo'] = fuzz.smf(ang.universe, np.pi/6, np.pi/2)

In [12]:
# Conjuntos difusos das saídas
guinada['Zero'] = fuzz.zmf(guinada.universe, 0, np.pi/6)
guinada['Modesta'] = fuzz.pimf(guinada.universe, np.pi/12, np.pi/4, np.pi/3, np.pi/2)
guinada['Media'] = fuzz.pimf(guinada.universe, np.pi/12, np.pi/4, np.pi/3, np.pi/2)
guinada['Grande'] = fuzz.pimf(guinada.universe, np.pi/12, np.pi/4, np.pi/3, np.pi/2)
guinada['Severa'] = fuzz.smf(guinada.universe, np.pi/6, np.pi/2)

vel['Lenta'] = fuzz.pimf(vel.universe, 0, 2, 4, 6)
vel['Rapida'] = fuzz.trapmf(vel.universe, [5, 6, 9, 10])
vel['MuitoRapida'] = fuzz.trapmf(vel.universe, [9, 10, 13, 14])
vel['Top'] = fuzz.pimf(vel.universe, 12, 13, 14, 15)

In [13]:
# Definição das regras
rules = [
    ctrl.Rule(dist['Pequena'] & ang['Pequeno'], [guinada['Severa'], vel['Lenta']]),
    ctrl.Rule(dist['Pequena'] & ang['Medio'], [guinada['Grande'], vel['Lenta']]),
    ctrl.Rule(dist['Pequena'] & ang['Amplo'], [guinada['Media'], vel['Rapida']]),
    ctrl.Rule(dist['Longa'] & ang['Pequeno'], [guinada['Grande'], vel['Lenta']]),
    ctrl.Rule(dist['Longa'] & ang['Medio'], [guinada['Media'], vel['Rapida']]),
    ctrl.Rule(dist['Longa'] & ang['Amplo'], [guinada['Modesta'], vel['MuitoRapida']]),
    ctrl.Rule(dist['MuitoLonga'] & ang['Pequeno'], [guinada['Media'], vel['Rapida']]),
    ctrl.Rule(dist['MuitoLonga'] & ang['Medio'], [guinada['Modesta'], vel['MuitoRapida']]),
    ctrl.Rule(dist['MuitoLonga'] & ang['Amplo'], [guinada['Zero'], vel['Top']]),
]

In [14]:
# Criando o sistema de controle
sistema_controle = ctrl.ControlSystem(rules)
simulador = ctrl.ControlSystemSimulation(sistema_controle)

In [15]:
# Exemplo de simulação
simulador.input['dist'] = 3.0
simulador.input['ang'] = np.pi/4
simulador.compute()

print(f"Guinada: {simulador.output['guinada']:.4f} rad")
print(f"Velocidade: {simulador.output['vel']:.4f} m/s")

Guinada: 0.9163 rad
Velocidade: 7.9936 m/s
